In [1]:
from config import config 
import pandas as pd
import random
import Levenshtein 
from tqdm import tqdm
import numpy as np
train =pd.read_csv("/home/wangjingqi/input/dataset/nesp/train_with_groups.csv")
print(len(train))
len(train.tm.unique())

31390


770

In [2]:
# test = pd.read_csv(config.test)
# tm = pd.read_csv("/home/wangjingqi/nesp/0.398.csv")
# base = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"
# test.loc[:,"dtm"] = tm.tm 
# test.loc[:,"seq0"]=[base]*len(test)
# test.loc[:,"seq1"]=test.protein_sequence
# test.to_csv(config.test)

In [3]:
test = pd.read_csv(config.test)
test.head()
edis=[]
for i in range(len(test)):
    edis.append(len(Levenshtein.editops(test.seq0[i],test.seq1[i])))
e = pd.DataFrame({"len":edis})
e.describe()

,len
count,2413.000000
mean,0.999586
std,0.020357
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [4]:
# train = train[train.pH.isin([7,8])].reset_index(drop=True)
# print(len(train))

In [5]:
# train = train.drop_duplicates(subset=['protein_sequence'], keep=False,).reset_index(drop=True)
# print(len(train))

In [6]:
train.loc[:,"seq_len"] = -1
for i in range(len(train)):
    train.loc [i,"seq_len"]= len(train.protein_sequence[i]) 
# train = train[train.seq_len<=config.seq_len].reset_index(drop=True)
print(len(train))

31390


In [7]:
train

,seq_id,protein_sequence,pH,data_source,tm,group,seq_len
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,-1,341
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,-1,286
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,-1,497
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,-1,265
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,-1,1451
...,...,...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8,-1,549
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2,-1,469
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6,-1,128
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7,-1,593


In [8]:
train.seq_len.describe()

count    31390.000000
mean       447.669513
std        640.728935
min          5.000000
25%        197.000000
50%        336.000000
75%        523.000000
max      32767.000000
Name: seq_len, dtype: float64

In [9]:
import numpy as np
seq0s = []
seq1s = []
gs = []
dtm = []
pH = []
def panduan(edi):
    e = []
    for i in edi:
        e += [i[1],i[2]]
    if len(edi) <=0: return False
    if np.max(e)-np.min(e)<config.seq_len: return True
    return False
def get_min(edi):
    e = []
    for i in edi:
        e += [i[1],i[2]]
    minp=np.max(e)-config.seq_len+1
    maxp = np.min(e)
    if minp <0 :
        minp = 0
    return   minp,maxp


for g,seqs in train.groupby("group"):
    if g == -1:
        continue
    seqs_list = list(seqs.protein_sequence)
    tm_list = list(seqs.tm)
    pH_list = list(seqs.pH)
    data_source_list = list(seqs.data_source)
    with tqdm(desc=f"group-{g}",unit='it', total=len(seqs_list)) as pbar:
        for i in range(len(seqs_list)):
                seq0 = seqs_list[i]
                for j in range(len(seqs_list)):
                    seq1 = seqs_list[j]
                    edi = Levenshtein.editops(seq0,seq1)
                    if len(edi) in [0,1] and pH_list[i]==pH_list[j] and data_source_list[i]==data_source_list[j]: #and   seq1!=seq0
                        if len(edi) ==1:
                            if  edi[0][0] in ['replace',"delete"]:
                                seq0s.append(seq0)
                                seq1s.append(seq1)
                                dtm.append(tm_list[j]-tm_list[i])
                                pH.append(pH_list[i])
            
                                gs.append(g)
                        elif len(edi) ==0:
                            if seq0 == seq1:
                                seq0s.append(seq0)
                                seq1s.append(seq1)
                                dtm.append(tm_list[j]-tm_list[i])
                                pH.append(pH_list[i])
                                gs.append(g)
                pbar.update()
                    
                    

group-407: 100%|██████████| 3/3 [00:00<00:00, 39568.91it/s]


In [10]:
pair_train = pd.DataFrame({"seq0":seq0s,"seq1":seq1s,"dtm":dtm,"pH":pH,"group":gs})

In [11]:
len(pair_train.seq0.unique())

2861

In [12]:
# pair_train = pair_train[pair_train.dtm != 0]
len(pair_train)

11603

In [13]:
pair_train.to_csv("/home/wangjingqi/input/dataset/nesp/pair_train.csv")

In [14]:
pair_train

,seq0,seq1,dtm,pH,group
0,MADEEALPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MADEEALPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,0.0,7.0,0
1,MADEEKAPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MADEEKAPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,0.0,7.0,0
2,MADEEKAPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MADEEKIPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,11.5,7.0,0
3,MADEEKAPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MADEEKNPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,11.0,7.0,0
4,MADEEKAPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MADEEKVPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,6.2,7.0,0
...,...,...,...,...,...
11598,MECPQCGHVSSEKAPKFCSECGQKLPSAATVQGDLKNDNTLVVSST...,MECPQCGHVSSEKAPKFCSECGQKLPSAATVQGDLKNDNTLVVSST...,3.5,7.0,406
11599,MECPQCGHVSSEKAPKFCSECGQKLPSAATVQGDLKNDNTLVVSST...,MECPQCGHVSSEKAPKFCSECGQKLPSAATVQGDLKNDNTLVVSST...,0.0,7.0,406
11600,RPAVT,RPAVT,0.0,7.0,407
11601,RPGSI,RPGSI,0.0,7.0,407
